# Data Cleaning and Exploration

## Part 1: Olympics Dataset
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals.

In [48]:
#Import libraries
import os
import pandas as pd

#Create file path and read file
olympics_file = os.path.join("Resources", "olympics.csv")
df_olympics = pd.read_csv(olympics_file, index_col=0, skiprows=1)

#Clean the columns names
for col in df_olympics.columns:
    if col[:2]=='01':
        df_olympics.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df_olympics.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df_olympics.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df_olympics.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df_olympics.index.str.split('\s\(') # split the index by '('

df_olympics.index = names_ids.str[0] # the [0] element is the country name (new index) 
df_olympics['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df_olympics = df_olympics.drop('Totals')
df_olympics.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


In [49]:
# Look at the values for the first country Afganistan
df_olympics.iloc[0]

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question
Which country has won the most gold medals in summer games?

In [50]:
def most_sum_gold(df):
    max_sum_gold = max(df["Gold"])
    Max_gold = df[df["Gold"] == max_sum_gold]
    return Max_gold.index[0]

most_sum_gold(df_olympics)

'United States'

### Question
Which country had the biggest difference between their summer and winter gold medal counts?


In [51]:
def max_gold_diff(df):
    my_data = df
    my_data["Gold Diff"] = abs(my_data["Gold"] - my_data["Gold.1"])
    max_golddiff = max(my_data["Gold Diff"])
    Max_diff = my_data[my_data["Gold Diff"] == max_golddiff]
    return Max_diff.index[0]

max_gold_diff(df_olympics)

'United States'

### Question
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only countries that have won at least 1 gold in both summer and winter are included.


In [52]:
def max_diff_rate(df):
    my_data = df[(df["Gold"] > 0) & (df["Gold.1"] > 0)]
    my_data["Gold Diff"] = abs(my_data["Gold"] - my_data["Gold.1"])/my_data["Gold.2"]
    max_golddiff = max(my_data["Gold Diff"])
    Max_diff = my_data[my_data["Gold Diff"] == max_golddiff]
    return Max_diff.index[0]

max_diff_rate(df_olympics)

C:\Users\Landrover\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


'Bulgaria'

### Medal Scoring
Below function creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function returns only the column with the countries as indexes.


In [53]:
def medal_score(df):
    my_data = df
    my_data["Points"] = my_data["Gold.2"] * 3 + my_data["Silver.2"] * 2 + my_data["Bronze.2"]
    return my_data["Points"]

medal_score(df_olympics).head()

Afghanistan      2
Algeria         27
Argentina      130
Armenia         16
Australasia     22
Name: Points, dtype: int64

## Part 2: US Census Dataset
For this section, I will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.


In [54]:
census_file = os.path.join("Resources", "census.csv")
census_df = pd.read_csv(census_file)
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861



### Question
Which state has the most counties in it?


In [55]:
def max_county_num(df):
    my_data = df.groupby("STNAME")
    Counties = pd.DataFrame(my_data["CTYNAME"].count())
    max_val = max(Counties["CTYNAME"])
    max_ctname = Counties[Counties["CTYNAME"] == max_val]
    return max_ctname.index[0]

max_county_num(census_df)

'Texas'

### Question
Which county has had the largest absolute change in population within the period 2010-2015? (population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, all six columns are considered.)


In [56]:
def max_pop_change(df):
    my_data = df
    rows = ["POPESTIMATE2010", "POPESTIMATE2011", "POPESTIMATE2012", "POPESTIMATE2013", "POPESTIMATE2014", "POPESTIMATE2015"]
    my_data["Pop Change"] = my_data.apply(lambda x: max(x[rows]) - min(x[rows]), axis = 1)
    max_change = max(my_data["Pop Change"])
    max_county = my_data[my_data["Pop Change"] == max_change]
    max_county = max_county.set_index("CTYNAME")
    return max_county.index[0]

max_pop_change(census_df)

'Texas'

### Question
In this datafile, the United States is broken up into four regions using the "REGION" column. 

This query finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE2014.

*The function returns a DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [57]:
def query_county(df):
    my_data = df[((df["REGION"]==1) | (df["REGION"]==2)) & (df["CTYNAME"].str.startswith("Washington")) & (df["POPESTIMATE2015"] > df["POPESTIMATE2014"])]
    return my_data[["STNAME","CTYNAME"]]

query_county(census_df)

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
